In [1]:
import numpy as np
from time import perf_counter as tpc

from teneva import cross, erank, getter, rand, truncate

np.random.seed(42)

In [2]:
def check(X_tst, Y_tst, Y, get, name):
    """Check the model Y for the given test data (X_tst, Y_tst)."""
    t = tpc()
    Z = np.array([get(x) for x in X_tst])
    t = tpc() - t
    e = np.linalg.norm(Z - Y_tst) / np.linalg.norm(Y_tst)

    print(f'------------------ {name}')
    print(f'Check time (sec) : {t:-8.3f}')
    print(f'Time 1-calc (ms) : {t/len(Z) * 1000:-8.3f}')
    print(f'Relative error   : {e:-8.2e}')
    print(f'Erank is         : {erank(Y):-8.2f}')

In [8]:
N = [9, 8, 8, 9, 8]  # Shape of the tensor
d = len(N)           # Dimension of the problem
M = 10000            # Number of test cases
nswp     = 6         # Sweep number
eps      = 1.E-6     # Desired accuracy
kickrank = 2         # Cross parameter
rf       = 1.5       # Cross parameter

In [9]:
def f(x):
    y = np.sum([x[i]**(i+1) for i in range(d)])
    y = np.sin(y)**2 / (1 + y**2)
    return  y

def f_vect(X):
    """Naively vectorized model function."""
    return np.array([f(x) for x in X])

In [10]:
X_tst = np.vstack([np.random.choice(N[i], M) for i in range(d)]).T
Y_tst = np.array([f(x) for x in X_tst])

In [11]:
Y0 = rand(N, 2)

In [12]:
t = tpc()
Y = cross(f_vect, Y0, nswp, kickrank=2, rf=1.5)
Y = truncate(Y, eps)
t = tpc() - t

get = getter(Y)
check(X_tst, Y_tst, Y, get, 'Cross own')
print(f'Build time (sec) : {t:-8.3f}')

------------------ Cross own
Check time (sec) :    0.415
Time 1-calc (ms) :    0.041
Relative error   : 4.11e-07
Erank is         :    13.80
Build time (sec) :    1.886


## Compare with ttpy

In [8]:
import tt                                   
from tt.cross.rectcross import cross as ttpy_cross

In [9]:
def solver_cro(f, Y0, nswp=10, eps=None, kickrank=2, rf=1.5):
    Y0 = tt.tensor.from_list(Y0)
    Y = ttpy_cross(myfun=f, x0=Y0, nswp=nswp, eps=1.E-16,
              eps_abs=0., kickrank=kickrank, rf=rf,
              verbose=True, stop_fun=None, approx_fun=None)
    if eps:
        Y = Y.round(eps)
    return tt.tensor.to_list(Y)  

In [10]:
t = tpc()
Yr_cro = solver_cro(f_vect, Y0, nswp, eps, kickrank, rf)
t = tpc() - t

get_cro = getter(Yr_cro)
check(X_tst, Y_tst, Yr_cro, get_cro, 'Cross')
print(f'Build time (sec) : {t:-8.3f}')

swp: 0/1 er_rel = 7.2e+02 er_abs = 7.2e+02 erank = 6.0 fun_eval: 936
swp: 1/1 er_rel = 3.1e-01 er_abs = 3.1e-01 erank = 9.6 fun_eval: 4264
------------------ Cross
Check time (sec) :    0.458
Time 1-calc (ms) :    0.046
Relative error   : 9.45e-03
Erank is         :     8.69
Build time (sec) :    0.186


---

## Tests

In [11]:
import tt

In [12]:
from teneva import erank

Z = tt.rand(np.array([5, 6, 7, 8, 9]))
Y = tt.tensor.to_list(Z)

print(f'Erank ttpy   : {Z.erank:-8.3f}')
print(f'Erank teneva : {erank(Y):-8.3f}')

Erank ttpy   :    2.000
Erank teneva :    2.000


In [13]:
from teneva import get


Z = tt.rand(np.array([5, 6, 7, 8, 9]))
Y = tt.tensor.to_list(Z)

x = [0, 1, 2, 1, 0]

y1 = Z[x]
y2 = get(Y, x)

print(f'Element ttpy   : {y1:-8.3f}')
print(f'Element teneva : {y2:-8.3f}')

Element ttpy   :    9.230
Element teneva :    9.230


In [14]:
from teneva import getter

Z = tt.rand(np.array([5, 6, 7, 8, 9]))
Y = tt.tensor.to_list(Z)

get = getter(Y)

x = np.array([0, 1, 2, 1, 0])

y1 = Z[x]
y2 = get(x)

print(f'Element ttpy   : {y1:-8.3f}')
print(f'Element teneva : {y2:-8.3f}')

Element ttpy   :   -0.999
Element teneva :   -0.999


In [15]:
from teneva import mean

Z = tt.rand(np.array([5, 6, 7, 8, 9]))
Y = tt.tensor.to_list(Z)

m1 = tt.sum(Z) / np.prod([G.shape[1] for G in Y])
m2 = mean(Y)

print(f'Mean ttpy   : {m1:-10.5f}')
print(f'Mean teneva : {m2:-10.5f}')

Mean ttpy   :    0.01657
Mean teneva :    0.01657


In [16]:
from teneva import mul

Z1 = tt.rand(np.array([5, 6, 7, 8, 9]))
Z2 = tt.rand(np.array([5, 6, 7, 8, 9]))
Y1 = tt.tensor.to_list(Z1)
Y2 = tt.tensor.to_list(Z2)

Z3 = Z1 * Z2
Y3 = mul(Y1, Y2)

n1 = Z3.norm()
n2 = tt.tensor.from_list(Y3).norm()

print(f'Norm ttpy   : {n1:-10.5f}')
print(f'Norm teneva : {n2:-10.5f}')

Norm ttpy   : 2580.77399
Norm teneva : 2580.77399


In [17]:
from teneva import norm

Z = tt.rand(np.array([5, 6, 7, 8, 9]))
Y = tt.tensor.to_list(Z)

n1 = Z.norm()
n2 = norm(Y)

print(f'Norm ttpy   : {n1:-10.5f}')
print(f'Norm teneva : {n2:-10.5f}')

Norm ttpy   :  294.88505
Norm teneva :  294.88505


In [18]:
from teneva import rand

def sf(k):
    return np.array([0.1] * k)

n = np.array([5, 6, 7, 8, 9])
r = 4
Z = tt.rand(n, r=r, samplefunc=sf)
Y = rand(n, r, sf)

n = (Z - tt.tensor.from_list(Y)).norm() / Z.norm()

print(f'Norm of error   : {n:-10.2e}')

Norm of error   :   3.57e-16


In [19]:
from teneva import recap

Z = tt.rand(np.array([5, 6, 7, 8, 9]))
Y = tt.tensor.to_list(Z)

m1 = tt.sum(Z)
m2 = recap(Y)

print(f'Mean ttpy   : {m1:-10.5f}')
print(f'Mean teneva : {m2:-10.5f}')

Mean ttpy   : 1722.55537
Mean teneva : 1722.55537


In [20]:
from teneva import truncate

Z = tt.rand(np.array([5, 6, 7, 8, 9]), r=10)
Y = tt.tensor.to_list(Z)

e = 1.E-2
Z = Z.round(e)
Y = truncate(Y, e)

n = (Z - tt.tensor.from_list(Y)).norm() / Z.norm()

print(f'Norm of error   : {n:-10.2e}')

Norm of error   :   3.38e-15


---